___
<img style="float: right; margin: 0px 0px 15px 15px;" 
src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios
-superiores-de-occidente.png" width="150px" height="100px" />


# MICROESTRUCTURAS Y SISTEMAS DE "TRADING"
# LABORATORIO 4

## Nombres: Ana Laura Correa , Mariana Lopez, Jocelyn Medel

## Fecha: 8 / Diciembre / 2019

#### Propósito: 
Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental).

#### Librerias:

In [1]:
#Importar librerias
import pandas as pd
import math 
import pandas as pd
import numpy as np
import warnings
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings("ignore")

#### Archivo de Precios Historicos

In [4]:
prices = pd.read_csv('precios_historicos_eurusd.csv',header=0, index_col=None, parse_dates=False)
prices['timestamp'] = [pd.to_datetime(prices.iat[i,0]).tz_localize('GMT') for i in range(len(prices))] # Timestamp a 'GMT'
prices.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


#### Archivo de Calendario Economico

In [8]:
data = pd.read_csv('calendario_economico.csv',header=0, index_col=None, parse_dates=False)
data.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,1/5/09 9:30,EUR,Sentix Investor Confidence,European Monetary Union,1,-34.4,-42.3,NaN
1,1/5/09 15:00,USD,Construction Spending (MoM),United States,1,-1.2,-0.4,-1.3
2,1/6/09 8:55,EUR,Markit Services PMI,Germany,1,46.6,46.4,46.4
3,1/6/09 9:00,EUR,Markit Services PMI,European Monetary Union,0,42.1,42.0,42.0
4,1/6/09 10:00,EUR,Consumer Price Index (YoY),European Monetary Union,2,1.6,2.1,1.8


In [9]:
Nonfarm = data.loc[data["Name"] == "Nonfarm Payrolls"].reset_index(drop=True) #Resetear el indice, para encontrar calendario económico
Nonfarm.consensus.fillna(Nonfarm.previous, inplace=True) #Formar Dataframe
Nonfarm["timestamp"] = pd.to_datetime(Nonfarm["timestamp"])
Nonfarm["timestamp"] = Nonfarm["timestamp"].dt.tz_localize('GMT')


### Escenarios del indicador

$A$ = $Actual$ ≥ $Consensus$ ≥ $Previous$

$B$ = $Actual$ ≥ $Consensus$ ≤ $Previous$

$C$ = $Actual$ ≤ $Consensus$ ≥ $Previous$

$D$ = $Actual$ ≤ $Consensus$ ≤ $Previous$

Donde:

$Actual$ : Es el valor actual del indicador.

$Consensus$ : Es el valor del indicador que se pronóstica.

$Previous$ : Es el valor anterior del indicador.

In [10]:
#Función  para encontrar los escenarios

def clase(x):
    """Param x:DataFrame
       Return x:DataFrame con columna de clase"""
    
    x["escenario"] = x["actual"]
    for i in range(len(x["actual"])):
        if x["actual"][i] >= x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "A"
        elif x["actual"][i] >= x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "B"
        elif x["actual"][i] < x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "C"
        elif x["actual"][i] < x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "D"
    
    return x

In [11]:
#Función para los periodos de 30 minutos antes y después de que se comunicó el índice 

def periodos(x,prices):
    xt = prices["timestamp"].isin(x["timestamp"])
    xt = [i for i in range(len(xt)) if xt[i] == True]
    t = [prices["timestamp"][i] for i in xt]
    xtprt = [prices.iloc[i-6:i+7] for i in xt]
    
    return xtprt,t

In [12]:
# Función para calcular las metricas señaladas 

def metrica(x,y,z):
    dire = lambda x: x.iloc[-1,-1]-x.iloc[int(len(x)/2),1]
    alc = lambda x: x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1]
    baj = lambda x: x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min()
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    
    return x

In [13]:
#Aplicamos las funciones ya explicadas 

Nonfarm = clase(Nonfarm)

In [14]:
Nonfarmper,Nonfarmt = periodos(Nonfarm,prices)

In [15]:
Nonfarm = metrica(Nonfarm,Nonfarmper,Nonfarmt)

In [65]:
Nonfarm.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-577.0,-584.0,-480.0,C,-0.01463,0.00004,0.01021,0.01999
1,2009-02-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-655.0,-577.0,-500.0,C,0.00373,0.00173,0.00275,0.00908
2,2009-03-06 13:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-651.0,-655.0,-625.0,C,0.00459,0.00503,0.00000,0.00906
3,2009-04-03 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-699.0,-651.0,-658.0,D,-0.00783,0.00187,0.00659,0.01112
4,2009-05-08 12:30:00+00:00,USD,Nonfarm Payrolls,United States,3,-504.0,-699.0,-620.0,A,0.00614,0.00567,0.00000,0.00917


In [67]:
Nonfarmper

[                    timestamp     open     high      low    close
 960 2009-01-09 13:00:00+00:00  1.36907  1.36958  1.36788  1.36788
 961 2009-01-09 13:05:00+00:00  1.36794  1.36932  1.36758  1.36932
 962 2009-01-09 13:10:00+00:00  1.36912  1.36943  1.36702  1.36840
 963 2009-01-09 13:15:00+00:00  1.36847  1.36942  1.36806  1.36936
 964 2009-01-09 13:20:00+00:00  1.36936  1.36976  1.36816  1.36916
 965 2009-01-09 13:25:00+00:00  1.36916  1.37073  1.36882  1.37073
 966 2009-01-09 13:30:00+00:00  1.37083  1.37521  1.36546  1.36952
 967 2009-01-09 13:35:00+00:00  1.36962  1.37334  1.36889  1.37088
 968 2009-01-09 13:40:00+00:00  1.37087  1.37170  1.36420  1.36540
 969 2009-01-09 13:45:00+00:00  1.36541  1.36645  1.35861  1.36122
 970 2009-01-09 13:50:00+00:00  1.36125  1.36249  1.35959  1.36200
 971 2009-01-09 13:55:00+00:00  1.36196  1.36392  1.36066  1.36066
 972 2009-01-09 14:00:00+00:00  1.36062  1.36090  1.35522  1.35620,
                      timestamp     open     high      low   

In [68]:
def escenarios(x,y,z):
    dire = lambda x: 1 if x.iloc[int(len(x)/2)]["close"] <= x.iloc[int(len(x)/2)]["open"]  else -1 
    alc = lambda x: x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1]
    baj = lambda x: x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min()
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    
    return x

In [70]:
df_escenarios = escenarios(Nonfarm,Nonfarmper,Nonfarmt)
df_escenarios= df_escenarios[["timestamp","escenario","direccion","alcista", "bajista", "volatilidad"]]
df_escenarios.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,C,1,0.00004,0.01021,0.01999
1,2009-02-06 13:30:00+00:00,C,1,0.00173,0.00275,0.00908
2,2009-03-06 13:30:00+00:00,C,-1,0.00503,0.00000,0.00906
3,2009-04-03 12:30:00+00:00,D,-1,0.00187,0.00659,0.01112
4,2009-05-08 12:30:00+00:00,A,-1,0.00567,0.00000,0.00917


In [56]:
A_count= list(df_escenarios["escenario"]).count("A")
B_count= list(df_escenarios["escenario"]).count("B")
C_count= list(df_escenarios["escenario"]).count("C")
D_count= list(df_escenarios["escenario"]).count("D")
print('The count of A is:', A_count)
print('The count of B is:', B_count)
print('The count of C is:', C_count)
print('The count of D is:', D_count)

The count of A is: 36
The count of B is: 36
The count of C is: 26
The count of D is: 31


In [81]:
A = df_escenarios.loc[:,"escenario"] == 'A'
df_A = df_escenarios.loc[A]
df_A.head()
kl=list(df_A["direccion"]).count(1)
kl

23

In [72]:
B = df_escenarios.loc[:,"escenario"] == 'B'
df_B = df_escenarios.loc[B]
df_B.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad
5,2009-06-05 12:30:00+00:00,B,-1,0.00697,0.00747,0.01779
11,2009-12-04 13:30:00+00:00,B,1,0.00000,0.01019,0.01665
14,2010-03-05 13:30:00+00:00,B,1,0.00000,0.00395,0.00536
16,2010-05-07 12:30:00+00:00,B,1,0.00000,0.00595,0.00829
20,2010-09-03 12:30:00+00:00,B,-1,0.00244,0.00064,0.00591


In [73]:
C = df_escenarios.loc[:,"escenario"] == 'C'
df_C = df_escenarios.loc[C]
df_C.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad
0,2009-01-09 13:30:00+00:00,C,1,0.00004,0.01021,0.01999
1,2009-02-06 13:30:00+00:00,C,1,0.00173,0.00275,0.00908
2,2009-03-06 13:30:00+00:00,C,-1,0.00503,0.00000,0.00906
9,2009-10-02 12:30:00+00:00,C,1,0.00000,0.00438,0.00686
13,2010-02-05 13:30:00+00:00,C,-1,0.00421,0.00000,0.00681


In [74]:
D = df_escenarios.loc[:,"escenario"] == 'D'
df_D = df_escenarios.loc[D]
df_D.head()

,timestamp,escenario,direccion,alcista,bajista,volatilidad
3,2009-04-03 12:30:00+00:00,D,-1,0.00187,0.00659,0.01112
6,2009-07-02 12:30:00+00:00,D,-1,0.00042,0.00210,0.00575
12,2010-01-08 13:30:00+00:00,D,-1,0.01119,0.00000,0.01510
18,2010-07-02 12:30:00+00:00,D,-1,0.00585,0.00000,0.00760
23,2010-12-03 13:30:00+00:00,D,-1,0.00900,0.00000,0.01109
